Та же оценка качества предсказаний, но с ограничением количества объектов внутри сектора в диаграмме агента

In [1]:
import pickle
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('..')
from rectangles_drawing import calculate_one_pixels_diffs, draw_all_rectangles
from osm_object import Image_OSM_object

from circle_diagram import calculate_sectors_count, create_circle_diagram
from circle_diagram import Image_circle_diagrams

from utils import read_pickle, dump_pickle

In [3]:
points = read_pickle('../dataset/100_random_points.pickle')
points_to_centers_6_8 = read_pickle('../dataset/100_points_to_cell_centers_48cells_6x8.pickle')
points_to_centers_3_4 = read_pickle('../dataset/100_points_to_cell_centers_12cells_3x4.pickle')

In [4]:
predictions_folder = '../dataset/predictions/'
predicts_osm_folder = 'predicts_osm'
rectangled_predicts_folder = 'rectangled_predicts_pixel'
objects_predicts_folder = 'rectangled_predicts_obj'

osm_objects_path = os.path.join(predictions_folder, predicts_osm_folder)

In [5]:
train_val_test = read_pickle("../dataset/train_val_test_split.pickle")
test_ids = train_val_test['test']

In [6]:
bigger_side_blocks_count = 8
sectors_count = calculate_sectors_count(bigger_side_blocks_count)
print("Sectors count:", sectors_count)

Sectors count: 616


In [7]:
def get_all_tags(img_osm_object):
    return [o.tag for o in img_osm_object.osm_objects if o.tag != 'ridge' and o.tag != 'valley' ]

In [8]:
result_ids = []

i = 0

bad_count_before_100 = 0
bad_count_after_100 = 0

while len(result_ids) < 100:
    img_id = test_ids[i]
    
    try:
        img_osm_object = read_pickle(os.path.join(osm_objects_path, img_id + '.pickle'))
    except Exception as e:
        print(e)
        # Дальше, чем 100, предсказаний нет
        break
    tags = get_all_tags(img_osm_object)
    
    if len(tags) < 2:
        print(f"{i}:, image: {img_id}, tags: {tags}")
        if i < 100:
            bad_count_before_100 += 1
        else:
            bad_count_after_100 += 1
    else:
        result_ids.append(img_id)
        
    i += 1

1:, image: 9501, tags: []
4:, image: 10196, tags: []
6:, image: 11093, tags: []
7:, image: 11980, tags: []
10:, image: 11410, tags: []
11:, image: 16382, tags: []
13:, image: 9437, tags: []
15:, image: 1006, tags: []
16:, image: 3800, tags: []
18:, image: 16832, tags: ['water']
20:, image: 18649, tags: []
23:, image: 16577, tags: []
24:, image: 14357, tags: []
25:, image: 14705, tags: []
26:, image: 17506, tags: []
27:, image: 14406, tags: []
29:, image: 9463, tags: []
30:, image: 14706, tags: []
33:, image: 18004, tags: []
34:, image: 4221, tags: []
36:, image: 21615, tags: []
38:, image: 11947, tags: []
39:, image: 16286, tags: []
43:, image: 21799, tags: []
44:, image: 13603, tags: []
46:, image: 1869, tags: []
47:, image: 11859, tags: ['water']
48:, image: 202, tags: []
50:, image: 18478, tags: []
55:, image: 12939, tags: []
56:, image: 18421, tags: []
57:, image: 8750, tags: []
58:, image: 14014, tags: []
59:, image: 20030, tags: ['peak']
60:, image: 19572, tags: []
61:, image: 13

In [9]:
print(len(result_ids))

42


In [10]:
def calculate_cells_probs(img_cds, agent_cd):
    """
    only fast version
    """
    penalties = []
    
    for i in range(len(img_cds.center_points)):
        penalties.append(img_cds.fast_diagrams[i].dist_to(agent_cd.fast_diagram))
        
    # Добавл. 1e-8 чтобы не было нулей
    similaryties = np.max(penalties) - penalties + 1e-8

    return similaryties / similaryties.sum()

In [11]:
def find_center_point_id(center_points, agent_point):
    for i in range(len(center_points)):
        if center_points[i] == agent_point:
            return i

In [12]:
def is_in_one_of_max_k(cell_probs, center_point_id, max_k):
    is_ok_for_different_k = []
    
    argsort = np.argsort(cell_probs)[::-1]
    for k in range(0, max_k):
        if argsort[k] == center_point_id:
            is_ok_for_different_k += [True for _ in range(max_k - k)]
            break
        else:
            is_ok_for_different_k.append(False)
        
    return is_ok_for_different_k

In [13]:
def cut_sectors(circle_diagram, max_len):
    for i in range(len(circle_diagram.sectors)):
        circle_diagram.sectors[i] = circle_diagram.sectors[i][:max_len]
    return circle_diagram

In [16]:
def calculate_precision_k_for_one(img_id, max_k):
    
    filename_pickle = img_id + '.pickle'
    
    img_osm = read_pickle(os.path.join(osm_objects_path, filename_pickle))
    img_osm.img_cutted_shape = (img_osm.img_cutted_shape[0], img_osm.img_cutted_shape[1], 3)
    img_cds = Image_circle_diagrams(img_osm, bigger_side_blocks_count, sectors_count, with_fast=True)
    
    is_in_k_one_image_all = []
    is_in_k_one_image_max_3 = []
    is_in_k_one_image_max_2 = []
    is_in_k_one_image_max_1 = []
    
    for agent_point in points:
        agent_point = agent_point[::-1]
        
        def _calc_one(max_in_sector=None):
            agent_cd = create_circle_diagram(sectors_count, agent_point, img_osm)
            if max_in_sector:
                agent_cd = cut_sectors(agent_cd, max_in_sector)
            agent_cd.make_fast()

            cell_probs = calculate_cells_probs(img_cds, agent_cd)

            center_point_id = find_center_point_id(img_cds.center_points, points_to_centers_6_8[agent_point[::-1]][::-1])

            is_in_k_one_point = is_in_one_of_max_k(cell_probs, center_point_id, max_k)
            
            return is_in_k_one_point
        
        is_in_k_one_point = _calc_one(max_in_sector=None)
        is_in_k_one_image_all.append(is_in_k_one_point)
        
        is_in_k_one_point_3 = _calc_one(max_in_sector=3)
        is_in_k_one_image_max_3.append(is_in_k_one_point_3)
        
        is_in_k_one_point_2 = _calc_one(max_in_sector=2)
        is_in_k_one_image_max_2.append(is_in_k_one_point_2)
        
        is_in_k_one_point_1 = _calc_one(max_in_sector=1)
        is_in_k_one_image_max_1.append(is_in_k_one_point_1)
        
        
    return is_in_k_one_image_all, is_in_k_one_image_max_3, is_in_k_one_image_max_2, is_in_k_one_image_max_1

In [18]:
%%time
is_in_k = []
is_in_k_max_3 = []
is_in_k_max_2 = []
is_in_k_max_1 = []

for img_id in result_ids[:100]:
    
    is_all, is_3, is_2, is_1 = calculate_precision_k_for_one(img_id, max_k=1)
    is_in_k += is_all
    is_in_k_max_3 += is_3
    is_in_k_max_2 += is_2
    is_in_k_max_1 += is_1
    
    if len(is_in_k) % 200 == 0:
        print(len(is_in_k))
        

200


../utils.py:39: RuntimeWarning: invalid value encountered in arccos
  return np.rad2deg(np.arccos(scalar_product / (norm_1 * norm_2)))


400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
CPU times: user 53min 2s, sys: 1.21 s, total: 53min 3s
Wall time: 53min 3s


In [19]:
is_in_k = np.array(is_in_k)

acc_k = []
for i in range(1):
    acc_k.append(is_in_k[:, i].sum() / is_in_k.shape[0])

In [20]:
is_in_k_max_3 = np.array(is_in_k_max_3)

acc_k_max_3 = []
for i in range(1):
    acc_k_max_3.append(is_in_k_max_3[:, i].sum() / is_in_k_max_3.shape[0])

In [21]:
is_in_k_max_2 = np.array(is_in_k_max_2)

acc_k_max_2 = []
for i in range(1):
    acc_k_max_2.append(is_in_k_max_2[:, i].sum() / is_in_k_max_2.shape[0])

In [22]:
is_in_k_max_1 = np.array(is_in_k_max_1)

acc_k_max_1 = []
for i in range(1):
    acc_k_max_1.append(is_in_k_max_1[:, i].sum() / is_in_k_max_1.shape[0])

In [23]:
print(acc_k)

[0.49666666666666665]


In [24]:
print(acc_k_max_3)

[0.4957142857142857]


In [25]:
print(acc_k_max_2)

[0.485]


In [26]:
print(acc_k_max_1)

[0.4030952380952381]
